## Load Dataset

In [1]:
import pyodbc
import mysql.connector
import dask.dataframe as dd
import pandas as pd
# from scripts.db_utils import create_ebay_connection
import os 
from dotenv import load_dotenv

load_dotenv()

def create_ebay_connection():
    """Create connection to the hosted SQL Server database for IP addresses."""
    try:
        connection = pyodbc.connect(
            f"DRIVER={{MySQL ODBC 9.1 Unicode Driver}};"
            f"SERVER={os.getenv('EBAY_DB_HOST')};"
            f"DATABASE={os.getenv('EBAY_DB_NAME')};"
            f"UID={os.getenv('EBAY_DB_USER')};"
            f"PWD={os.getenv('EBAY_DB_PASSWORD')}"
        )
        return connection
    except pyodbc.Error as e:
        print(f"Error connecting to hosted database: {e}")
        return None
# Database connection parameters
conn = create_ebay_connection()

query = "SELECT * FROM tbl_iptrace_user_activity where ams_trans_rsn_cd=0"


def fetch_data_in_chunks(conn, query, chunk_size=100000):
    """
    Fetch data from the database in chunks, then convert to Dask DataFrame.
    """
    offset = 0
    while True:
        # Modify the query to include pagination using LIMIT and OFFSET for chunks
        paginated_query = f"{query} LIMIT {chunk_size} OFFSET {offset}"
        
        # Fetch data in chunks using pandas (it handles column and row alignment internally)
        df_chunk = pd.read_sql_query(paginated_query, conn)
        
        if df_chunk.empty:
            break
        
        # Convert to Dask DataFrame
        dask_df = dd.from_pandas(df_chunk, npartitions=4)
        
        yield dask_df
        
        offset += chunk_size


dask_chunks = []

# Fetch chunks and store them in the list
for dask_df_chunk in fetch_data_in_chunks(conn, query, chunk_size=100000):
    dask_chunks.append(dask_df_chunk)

# Concatenate all chunks into a single Dask DataFrame
full_dask_df = dd.concat(dask_chunks)


# Show the result
print(full_dask_df.head())


C:\Users\ramegupta\AppData\Local\Temp\ipykernel_22208\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_22208\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_22208\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\

   id     CLNT_RMT_IP  ROI_CLICK_EVENT_TS  \
0   1  77.111.247.168 2020-08-02 18:37:01   
1   2   66.249.73.101 2020-08-21 03:36:38   
2   3  77.111.247.129 2020-08-21 03:36:38   
3   4   66.249.73.101 2020-08-21 03:36:33   
4   5  77.111.247.129 2020-08-21 03:36:33   

                                          BRWSR_NAME  ams_trans_rsn_cd  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   

  ams_pblshr_id    buyer_id  is_vpn            click_id  
0    5574672411   valen_949       1  209245484389035777  
1    5574672411  morbanisaf       1  209453736892211586  
2    5574672411  morbanisaf       1  209453736892211586  
3    5574672411  morbanisaf       1  209453736892211

In [42]:
# full_dask_df = full_dask_df.drop(columns=['id', 'ams_trans_rsn_cd', 'is_vpn'])
full_dask_df.tail()


,CLNT_RMT_IP,ROI_CLICK_EVENT_TS,BRWSR_NAME,ams_pblshr_id,buyer_id,click_id
54238,172.226.114.69,2024-09-05 11:04:49,ebayUserAgent/eBayIOS;6.174.0;iOS;17.6.1;Apple...,5575612316,srw38,226172465646404352
54239,208.115.224.233,2024-09-29 06:41:18,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5575532731,katzenklo_16,226442175813927936
54240,138.199.29.213,2024-10-09 03:15:37,ebayUserAgent/eBayAndroid;6.179.0;Android;14;s...,5575319207,guyver13,226553867601852672
54241,172.225.240.192,2024-10-27 03:30:27,Mozilla/5.0 (iPhone; CPU iPhone OS 18_1 like M...,5575376664,winde_-129,226757201610955392
54242,172.226.114.83,2024-11-03 04:57:09,Mozilla/5.0 (iPhone; CPU iPhone OS 18_0_1 like...,5575791307,monika_pl,226837779948133120


In [30]:
unique_count = full_dask_df['click_id'].nunique().compute()

# Display the result
print(f"Unique value count in 'click_id': {unique_count}")


Unique value count in 'click_id': 918884


In [31]:
print('no. of rows :', full_dask_df.shape[0].compute())

no. of rows : 1254243


## Prepare test(300k rows) and train(100k rows) data

In [43]:
import dask.dataframe as dd

# Convert the Dask DataFrame to Pandas DataFrame
full_df_pandas = full_dask_df.compute().reset_index(drop=True)

# Sample 300k rows for the training set using Pandas
train_df_pandas = full_df_pandas.sample(n=300000, random_state=42)

# Remove the sampled rows for training set from the original DataFrame to get the remaining rows
remaining_df_pandas = full_df_pandas[~full_df_pandas.index.isin(train_df_pandas.index)]

# Sample 100k rows for the testing set from the remaining rows
test_df_pandas = remaining_df_pandas.sample(n=100000, random_state=42)

# Check the sizes of the sampled DataFrames
train_size = train_df_pandas.shape[0]
test_size = test_df_pandas.shape[0]

print(f"Training set size: {train_size}")
print(f"Testing set size: {test_size}")





Training set size: 300000
Testing set size: 100000


In [45]:
train_df_pandas.to_csv('../data/csv/train_set.csv', index=False)
test_df_pandas.to_csv('../data/csv/test_set.csv', index=False)


## Load train dataset and implement preprocessing

In [ ]:
import pandas as pd

data = pd.read_csv('../data/csv/train_set.csv')